# Figures and tables III


This file is part of the replication code for: 

Offshore Profit Shifting and Aggregate Measurement: Balance of Payments, Foreign Investment, Productivity, and the Labor Share by Guvenen, Mataloni Jr., Rassier, and Ruhl. 

This version: February 21, 2022


This file creates all of the tables in the paper except for tables 5 and, which are created in `2-analysis-I.ipynb` and table 3, which is created in `(7) Hines and Rice 1982 AER Replication.sas` in the `0-confidential-data-replication-files/USDIA` folder.

In [1]:
import pandas as pd

def add_after(text, beginswith, toinsert):
    '''Adds a row to a latex table. 
    Inserts the string `toinsert` into the table string `text` 
    after the row that begins with `beginswith`.'''
    ss = text.splitlines()
    
    ii = '0'
    for i, l in enumerate(ss):
        if l.startswith(beginswith) == True:
            ii = i
    
    # This will throw an error if beginswith is not found.
    ss.insert(ii+1, toinsert)
    return '\n'.join(ss)

## Table 1

In [2]:
ctys = ['All Countries Total', 'Canada', 'Ireland', 'Luxembourg', 'Netherlands', 'Switzerland',
        'Barbados', 'Bermuda', 'United Kingdom Islands, Caribbean', 'Hong Kong', 'Singapore']

missing = ['(*)', 'G', 'A', 'n.s.', 'H', '(D)']

# Employees are in thousands. Express them in people. 
emp = pd.read_csv('../1-raw-data/usdia-employees-2012.csv', header = 5, nrows=234, na_values=missing, converters = {0: str.strip}, index_col=[0])
emp.columns = ['emp']
emp = emp*1000

ast = pd.read_csv('../1-raw-data/usdia-total-assets-2012.csv', header = 5, nrows=234, na_values=missing, converters = {0: str.strip}, index_col=[0])
ast.columns = ['assets']

comp = pd.read_csv('../1-raw-data/usdia-comp-2012.csv', header = 5, nrows=234, na_values=missing, converters = {0: str.strip}, index_col=[0])
comp.columns = ['comp']

ppe = pd.read_csv('../1-raw-data/usdia-ppe-2012.csv', header = 5, nrows=234, na_values=missing, converters = {0: str.strip}, index_col=[0])
ppe.columns = ['ppe']

t1 = pd.merge(left=emp, right=ast, left_index=True, right_index=True, how = 'inner')
t1 = pd.merge(left=t1, right=comp, left_index=True, right_index=True, how = 'inner')
t1 = pd.merge(left=t1, right=ppe, left_index=True, right_index=True, how = 'inner')

In [3]:
t1 = t1.loc[ctys].copy()
t1['Employment'] = t1['assets']/t1['emp']
t1['Compensation'] = t1['assets']/t1['comp']
t1['PPE'] = t1['assets']/t1['ppe']
t1 = t1[['PPE', 'Compensation', 'Employment']]

t1 = t1.rename(index={'All Countries Total':'World', 'United Kingdom Islands, Caribbean':'U.K.I., Caribbean'})

In [4]:
s = t1.to_latex(float_format="%.1f", column_format='lSSS', caption='Assets in U.S.-owned foreign affiliates, 2012', escape=False)
s = add_after(s, 'Canada', '[1ex]')
s = add_after(s, 'Switzerland', '[1ex]')
s = add_after(s, 'U.K.I., Caribbean', '[1ex]')

print(s)
fout = open('../4-tables/table1.tex', 'w')
fout.write(s)
fout.close()

\begin{table}
\centering
\caption{Assets in U.S.-owned foreign affiliates, 2012}
\begin{tabular}{lSSS}
\toprule
{} &    PPE &  Compensation &  Employment \\
\midrule
World             &   16.8 &          39.0 &         1.8 \\
Canada            &    6.4 &          21.2 &         1.2 \\
[1ex]
Ireland           &   20.0 &         142.7 &        10.9 \\
Luxembourg        & 1019.6 &        1380.0 &       121.6 \\
Netherlands       &   97.7 &         115.3 &         8.7 \\
Switzerland       &   59.9 &          60.0 &         7.7 \\
[1ex]
Barbados          &   41.8 &        1444.7 &        43.3 \\
Bermuda           &  130.8 &        1475.5 &       155.8 \\
U.K.I., Caribbean &  101.2 &        3330.2 &       199.8 \\
[1ex]
Hong Kong         &   40.3 &          39.3 &         2.7 \\
Singapore         &   18.6 &          50.3 &         3.1 \\
\bottomrule
\end{tabular}
\end{table}


## Table 2

Grab income on usdia by country. 

In [5]:
ctys = ['Netherlands', 'Bermuda', 'Ireland', 'Luxembourg', 'United Kingdom Islands, Caribbean[2]',
        'Switzerland', 'Singapore', 'Japan', 'Germany', 
        'United Kingdom', 'France', 'China', 'Italy', 'Canada']

usdia_inc = pd.read_excel('../1-raw-data/usdiainc_country.xls', header=6, nrows=253, na_values=['n.s.', '...', '(D)', '(*)', '---','--', '----'],
                       index_col='Unnamed: 0')
usdia_inc.index = usdia_inc.index.str.strip()

usdia_inc.head(1)

,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
All Countries Total,24828.0,27260.0,30746.0,28778.0,31040.0,40028.0,52057.0,53929.0,58004.0,52087.0,...,350237.0,392954.0,340039.0,417605.0,448235.0,438089.0,448890.0,457122.0,433333.0,430948.0


We need the income adjustment factor from the aggregate public data.

In [6]:
agg_data = pd.read_csv('../3-intermediate-files/aggregate.csv', index_col=['year'])
iaf16 = agg_data.loc[2016, 'income_adj_factor']

Load the country-level adjustments. The total adjustment is the adjustment from the confidential data plus (minus wrt the sign convention) the net income payments times the income adjustment factor. 

In [7]:
cty_adj = pd.read_excel('../0-confidential-data-replication-files/USDIA/OutputAdjNetCountry2016.xlsx', 
                   index_col='CountryName')
cty_adj = cty_adj.rename(index={'UK Islands - Caribbean':'United Kingdom Islands, Caribbean[2]',
                                'UK': 'United Kingdom'})

temp = pd.concat([cty_adj.loc[ctys], usdia_inc.loc[ctys, '2016']], axis=1)

temp['total adjustment earn'] = (temp['adjwt3s'])*iaf16

total_adj_earn = cty_adj['adjwt3s'].sum()*iaf16

temp['share adj earn'] = temp['total adjustment earn']/total_adj_earn*100
temp['share income'] = ((temp['adjwt3s']-temp['nips'])/(temp['2016'])*100*1000).abs()
temp.head(1)

,adjcomps,adjsales,adjrdstks,adjppes,adjwt3s,nips,2016,total adjustment earn,share adj earn,share income
Netherlands,-56.056532,-52.111587,-54.030339,-54.69904,-56.310608,-11.083156,70098.0,-59.12203,42.142011,64.520318


Arrange everything to output. 

In [8]:
pos = ['Netherlands', 'Bermuda', 'Ireland', 'Luxembourg', 
                  'United Kingdom Islands, Caribbean[2]', 'Switzerland', 'Singapore']
neg = ['Germany', 'France', 'United Kingdom',  'Canada',    'Italy', 'China','Japan']

t2 = temp.loc[pos, ['share adj earn', 'share income']]
negs = temp.loc[neg, ['share adj earn', 'share income']].reset_index()
negs['share adj earn'] = negs['share adj earn']*(-1)
negs = negs.rename(index=dict(zip(range(0,7),pos)))
t2 = pd.concat( [t2, negs], axis=1).reset_index()

t2.loc[t2['level_0']=='United Kingdom Islands, Caribbean[2]', 'level_0'] = 'U.K.I., Caribbean'
t2['index'] = '\hspace{1ex}' + t2['index']
t2['level_0'] = '\hspace{1ex}' + t2['level_0']

In [9]:

formatters=[lambda x: '%s' % x, lambda x: '%.1f' % x, lambda x: '%.1f' % x, 
            lambda x: '%s' % x, lambda x: '%.2f' % x, lambda x: '%.1f' % x]
            
s = t2.to_latex(formatters = formatters, column_format='lcclc', index=False, escape=False)

print(s)
fout = open('../4-tables/table2.tex', 'w')
fout.write(s)
fout.close()

\begin{tabular}{lcclc}
\toprule
                      level_0 & share adj earn & share income &                      index & share adj earn & share income \\
\midrule
      \hspace{1ex}Netherlands &           42.1 &         64.5 &        \hspace{1ex}Germany &           7.96 &        211.7 \\
          \hspace{1ex}Bermuda &           30.0 &         81.8 &         \hspace{1ex}France &           5.37 &        242.5 \\
          \hspace{1ex}Ireland &           23.1 &         54.3 & \hspace{1ex}United Kingdom &           4.58 &         26.8 \\
       \hspace{1ex}Luxembourg &           17.8 &         65.2 &         \hspace{1ex}Canada &           4.01 &         26.8 \\
\hspace{1ex}U.K.I., Caribbean &           21.0 &         84.2 &          \hspace{1ex}Italy &           3.68 &        296.3 \\
      \hspace{1ex}Switzerland &           12.1 &         46.9 &          \hspace{1ex}China &           3.08 &         34.6 \\
        \hspace{1ex}Singapore &           10.6 &         52.5 &          \hsp

## Table 4

In [10]:
t4 = pd.read_excel('../0-confidential-data-replication-files/USDIA/OutputAdjNetIndustry.xlsx', 
                   usecols=['Year','IEDindPar','adjwt3s'])
t4 = t4[t4['Year']==2016]

ied = pd.read_csv('../1-raw-data/author-created-concordances/ied-codes.csv')
t4 = pd.merge(left=t4, right=ied, left_on='IEDindPar', right_on='IED Code')

t4['total adjustment'] = t4['adjwt3s']*iaf16
total_adj = t4['total adjustment'].sum()
t4 = t4[['NAICS', 'NAICS Description', 'total adjustment']].groupby(['NAICS', 'NAICS Description']).sum().sort_values('total adjustment')
t4['share of adjustment'] = t4['total adjustment']/total_adj*100
t4 = t4.reset_index()
t4.head(1)

,NAICS,NAICS Description,total adjustment,share of adjustment
0,33,"Metals, machinery, computers, electrical equip...",-50.956455,36.321613


Get the value added by industry data. A few industry names change over the sample, so they need to be tweaked to match the concordance.

In [11]:
nva_ind = pd.read_csv('../1-raw-data/va-by-industry-97-16.csv', header=4, nrows=98)
nva_ind = nva_ind[['Unnamed: 1', '2016']]
nva_ind['Description nva'] = nva_ind['Unnamed: 1'].str.strip()

reps = {'Publishing industries, except internet (includes software)':'Publishing industries',
       'Data processing, internet publishing, and other information services':'Information and data processing services'}

nva_ind['Description nva'] = nva_ind['Description nva'].replace(reps)

nva_ind = pd.merge(left=nva_ind, right=ied, left_on='Description nva', right_on='Description', how='inner')
nva_ind = nva_ind.groupby('NAICS').sum().reset_index()

t4 = pd.merge(left=t4, right=nva_ind[['NAICS', '2016']], on='NAICS')
t4['share of va'] = t4['total adjustment']/t4['2016']*100*(-1)

Print out the table. 

In [12]:
pretty_names = {'Metals, machinery, computers, electrical equipment, motor vehicles, furniture':'Computers, electrical equip., motor vehicles',
                'Wood, paper, printing, petroleum, chemicals, plastics':'Petroleum, chemicals, plastics, wood, paper',
                'Professional and business services':'Legal, computer sys. design, scientific R\&D',
                'Information':'Publishing, information, data processing',
                'Finance and insurance': 'Finance, insurance',
               'Real estate and rental and leasing':'Real estate, leasing',
               'Administrative and waste management services':'Administrative services',
               'Transportation and warehousing':'Transportation, warehousing',
               'Educational services, health care, and social assistance':'Education, health care',
               'Arts, entertainment, recreation, accommodation, and food services':'Arts, entertainment, accommodation'}

t4['NAICS Description'] = t4['NAICS Description'].replace(pretty_names)
t4['Key industries'] = t4['NAICS'].astype(str) + ' ' + t4['NAICS Description']
t4 = t4.drop(['NAICS Description', 'NAICS', 'total adjustment', '2016'], axis=1)

t4 = t4[['Key industries', 'share of adjustment', 'share of va']]

In [13]:
s = t4.to_latex(float_format='%.2f', column_format='lSS', index=False, escape=False)
print(s)
fout = open('../4-tables/table4.tex', 'w')
fout.write(s)
fout.close()

\begin{tabular}{lSS}
\toprule
                                 Key industries &  share of adjustment &  share of va \\
\midrule
33 Computers, electrical equip., motor vehicles &                36.32 &         4.65 \\
 32 Petroleum, chemicals, plastics, wood, paper &                27.01 &         5.30 \\
54 Legal, computer sys. design, scientific R\&D &                 8.28 &         0.83 \\
    51 Publishing, information, data processing &                 8.12 &         1.17 \\
                          52 Finance, insurance &                 5.58 &         0.53 \\
                     31 Food, textiles, apparel &                 3.24 &         1.55 \\
                                      21 Mining &                 3.22 &         2.07 \\
                        53 Real estate, leasing &                 2.31 &         0.13 \\
           80 Other services, except government &                 2.10 &         0.74 \\
                                   22 Utilities &                 1.94 

## Table 7

In [14]:
stata = pd.read_excel('../0-confidential-data-replication-files/FDIUS/TableForPaper.xlsx', 
                      sheet_name='Stata Output', index_col='Year')

rd = pd.read_excel('../0-confidential-data-replication-files/FDIUS/TableForPaper.xlsx', 
                      sheet_name='BEA AMNE FDIUS RD', header=6, nrows=1, index_col=[0]).transpose()
rd.index = rd.index.astype(int)
rd = rd.rename(columns={'All Industries Total':'total rd'})

emp = pd.read_excel('../0-confidential-data-replication-files/FDIUS/TableForPaper.xlsx', 
                      sheet_name='BEA AMNE FDIUS Emp', header=6, nrows=1, index_col=[0]).transpose()
emp.index = emp.index.astype(int)
emp = emp.rename(columns={'All Industries Total':'total emp'})

t7 = pd.merge(left=stata, right=rd, left_index=True, right_index=True)
t7 = pd.merge(left=t7, right=emp, left_index=True, right_index=True)

t7['rd share'] = t7['RD']/t7['total rd']/1000
t7['emp share'] = t7['Employees']/t7['total emp']/1000
t7['shifted profit'] = t7['AdjFA']/1000000

t7.head(1)

,N,RD,Employees,AdjFA,total rd,total emp,rd share,emp share,shifted profit
Year,,,,,,,,,
2008,137,18461505,759509,2.093656e+06,40727,5636.2,0.453299,0.134756,2.093656


We need the aggregate adjustment to USDIA, too. 

In [15]:
usdia = pd.read_excel('../0-confidential-data-replication-files/USDIA/OutputAdjNet.xlsx', 
                      usecols=['Year', 'adjwt3s'])
usdia = pd.merge(left=usdia, right=agg_data['income_adj_factor'], left_on='Year', right_index=True)

usdia['usdia shifted profit'] = usdia['income_adj_factor']*usdia['adjwt3s']*(-1)

t7 = pd.merge(left=t7, right=usdia, left_index = True, right_on='Year', how='left')
t7.head(1)

,N,RD,Employees,AdjFA,total rd,total emp,rd share,emp share,shifted profit,Year,adjwt3s,income_adj_factor,usdia shifted profit
16,137,18461505,759509,2.093656e+06,40727,5636.2,0.453299,0.134756,2.093656,2008,-176.418809,1.032865,182.21681


Set up the table for printing. 

In [16]:
t7 = t7[['Year', 'N', 'rd share', 'emp share', 'shifted profit', 'usdia shifted profit']]

formatters=[lambda x: '%.0f' % x, lambda x: '%.0f' % x, lambda x: '%.2f' % x, 
            lambda x: '%.2f' % x, lambda x: '%.1f' % x, lambda x: '%.1f' % x]
            
s = t7.to_latex(formatters = formatters, column_format='lccccc', index=False, escape=False)

print(s)
fout = open('../4-tables/table7.tex', 'w')
fout.write(s)
fout.close()

\begin{tabular}{lccccc}
\toprule
Year &   N & rd share & emp share & shifted profit & usdia shifted profit \\
\midrule
2008 & 137 &     0.45 &      0.13 &            2.1 &                182.2 \\
2012 & 136 &     0.43 &      0.13 &           12.3 &                144.0 \\
2015 & 127 &     0.32 &      0.11 &            3.5 &                156.5 \\
\bottomrule
\end{tabular}



## Table C1

In [17]:
ctys[ctys.index('United Kingdom Islands, Caribbean[2]')] = 'UK Islands - Caribbean'
ctys[ctys.index('United Kingdom')] = 'UK'

In [18]:
cty_adj = pd.read_excel('../0-confidential-data-replication-files/USDIA/OutputAdjNetCountry2016.xlsx', index_col='CountryName')

oth = pd.DataFrame(cty_adj.loc[~cty_adj.index.isin(ctys)].sum(), columns=['Other countries']).transpose()
tot = pd.DataFrame(cty_adj.sum(),                                columns=['All countries']).transpose()

c1 = pd.concat([tot, cty_adj.loc[ctys].sort_index(), oth], axis=0)
c1 = c1[['adjwt3s', 'adjcomps', 'adjppes', 'adjrdstks']]*iaf16

c1.columns = ['{Weighted}', '{Only compensation}', '{Only PPE}', '{Only R\&D}']
c1 = c1.rename(index={'UK':'United Kingdom', 'UK Islands - Caribbean':'U.K.I., Caribbean'})

In [19]:
s = c1.to_latex(float_format='%.1f', column_format='lSSSS', escape=False)

print(s)
fout = open('../4-tables/tablec1.tex', 'w')
fout.write(s)
fout.close()

\begin{tabular}{lSSSS}
\toprule
{} &  {Weighted} &  {Only compensation} &  {Only PPE} &  {Only R\&D} \\
\midrule
All countries     &      -140.3 &               -125.1 &      -120.8 &       -167.2 \\
Bermuda           &       -42.1 &                -43.0 &       -42.2 &        -40.5 \\
Canada            &         5.6 &                  7.5 &         5.9 &          1.7 \\
China             &         4.3 &                  7.2 &         7.1 &         -1.3 \\
France            &         7.5 &                 12.0 &         5.8 &          3.5 \\
Germany           &        11.2 &                 13.5 &         8.4 &         10.9 \\
Ireland           &       -32.5 &                -43.8 &        -8.1 &        -44.4 \\
Italy             &         5.2 &                  6.5 &         6.4 &          2.3 \\
Japan             &         3.0 &                  5.8 &         1.8 &          1.1 \\
Luxembourg        &       -25.0 &                -25.5 &       -25.2 &        -23.1 \\
Netherlands      

## Table C2

In [20]:
indirect = pd.read_excel('../0-confidential-data-replication-files/USDIA/OutputAdjIndirect.xlsx',
                   index_col='Year', usecols=['Year', 'adjwt3s'])
indirect['adjwt3s'] = indirect['adjwt3s']*(-1)*agg_data['income_adj_factor']
indirect.columns = ['Indirectly-held affiliates']

direct = pd.read_excel('../0-confidential-data-replication-files/USDIA/OutputAdjDirect.xlsx',
                   index_col='Year', usecols=['Year', 'adjwt3s'])
direct['adjwt3s'] = direct['adjwt3s']*(-1)*agg_data['income_adj_factor']
direct.columns = ['Directly-held affiliates']

c2 = pd.merge(left=direct, right=indirect, right_index=True, left_index=True, how='inner')
c2['Net adjustment'] = c2['Directly-held affiliates'] + c2['Indirectly-held affiliates']

c2 = c2.reindex(range(1982,2017)).interpolate()
c2.head(1)

,Directly-held affiliates,Indirectly-held affiliates,Net adjustment
Year,,,
1982,16.41182,-2.231101,14.180719


In [21]:
s = c2.to_latex(float_format='%.1f', column_format='lSSS', escape=False)

print(s)
fout = open('../4-tables/tablec2.tex', 'w')
fout.write(s)
fout.close()

\begin{tabular}{lSSS}
\toprule
{} &  Directly-held affiliates &  Indirectly-held affiliates &  Net adjustment \\
Year &                           &                             &                 \\
\midrule
1982 &                      16.4 &                        -2.2 &            14.2 \\
1983 &                      18.5 &                        -3.2 &            15.4 \\
1984 &                      20.7 &                        -4.1 &            16.6 \\
1985 &                      22.8 &                        -5.0 &            17.8 \\
1986 &                      24.9 &                        -5.9 &            19.0 \\
1987 &                      27.0 &                        -6.9 &            20.2 \\
1988 &                      29.2 &                        -7.8 &            21.4 \\
1989 &                      31.3 &                        -8.7 &            22.6 \\
1990 &                      32.4 &                        -9.6 &            22.8 \\
1991 &                      33.5 &    

## Numbers for the draft

In [22]:
adj2015 = pd.read_excel('../0-confidential-data-replication-files/USDIA/OutputAdjNetCountry2015.xlsx', 
                   index_col='CountryName')
adj2015 = adj2015*agg_data.loc[2015,'income_adj_factor']

x = {}

# Section 1: Introduction
x[1.09] = ['asset-ppe ratio Canada', t1.loc['Canada', 'PPE'], '{:.1f}']
x[1.10] = ['avg asset-ppe ratio for euro tax havens', t1.loc[['Ireland', 'Luxembourg', 'Netherlands', 'Switzerland'], 'PPE'].mean(), '{:.0f}']
x[1.11] = ['avg asset-ppe ratio for caribb tax havens', t1.loc[['Barbados', 'Bermuda', 'U.K.I., Caribbean'], 'PPE'].mean(), '{:.0f}']
x[1.17] = ['2016 Netherland share of adjustment', t2.iloc[0,1]/100, '{:.0%}']
x[1.18] = ['2016 Bermuda share of adjustment', t2.iloc[1,1]/100, '{:.0%}']

x[1.43] = ['2012 directly held affilates ajustment ($bil)', c2.loc[2012,'Directly-held affiliates'], '{:.0f}']

# Section 4: 
x[4.13] = ['2016 Netherland share of adjustment', t2.iloc[0,1]/100, '{:.1%}']
x[4.14] = ['2016 Ireland share of adjustment', t2.iloc[2,1]/100, '{:.1%}']
x[4.15] = ['2016 NAICS 32 share of adjustment', t4.iloc[1, 1]/100, '{:.0%}']
x[4.16] = ['2016 NAICS 51+54 share of adjustment', (t4.iloc[3,1]+t4.iloc[2,1])/100, '{:.0%}']          
x[4.17] = ['2016 NAICS 52 share of adjustment', t4.iloc[4, 1]/100, '{:.1%}']

x[4.19] = ['2015 Ireland adjustment ($bil)', adj2015.loc['Ireland', 'adjwt3s']*(-1), '{:.1f}']
x[4.20] = ['2015 Luxembourg adjustment ($bil)', adj2015.loc['Luxembourg', 'adjwt3s']*(-1), '{:.1f}']
x[4.21] = ['2015 Netherlands adjustment ($bil)', adj2015.loc['Netherlands', 'adjwt3s']*(-1), '{:.1f}']
x[4.22] = ['2015 Switzerland adjustment ($bil)', adj2015.loc['Switzerland', 'adjwt3s']*(-1), '{:.1f}']
x[4.23] = ['2016 Ireland weighted adjustment ($bil)', c1.loc['Ireland', '{Weighted}']*(-1), '{:.1f}']
x[4.24] = ['2016 Ireland comp only adjustment ($bil)', c1.loc['Ireland', '{Only compensation}']*(-1), '{:.1f}']
x[4.25] = ['2016 Ireland red only adjustment ($bil)', c1.loc['Ireland', '{Only R\&D}']*(-1), '{:.1f}']
x[4.26] = ['2016 Ireland ppe only adjustment ($bil)', c1.loc['Ireland', '{Only PPE}']*(-1), '{:.1f}']

# Section 6

x[6.00] = ['sample firms avg share of RD spending', t7['rd share'].mean(), '{:.0%}']
x[6.01] = ['sample firms avg share of employment', t7['emp share'].mean(), '{:.0%}']
x[6.02] = ['increase in gdp from fdius proft shifting 2008 ($bil)',t7.loc[t7['Year']==2008,'shifted profit'].values[0], '{:.1f}']
x[6.03] = ['increase in gdp from fdius proft shifting 2012 ($bil)',t7.loc[t7['Year']==2012,'shifted profit'].values[0], '{:.1f}']
x[6.04] = ['increase in gdp from fdius proft shifting 2015 ($bil)',t7.loc[t7['Year']==2015,'shifted profit'].values[0], '{:.1f}']

pd.DataFrame.from_dict(x, columns=['Description', 'Number', 'Format'], orient='index').to_csv('../3-intermediate-files/numbers-in-text-III.csv')